In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
import random

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
tf.set_random_seed(777) # reproducibility

In [5]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [6]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [7]:
# input place holders
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1]) # img 28x28x1 (black/white)
Y = tf.placeholder(tf.float32, [None, 10])

In [8]:
# L1 ImgIn shape=(?, 28, 28, 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev = 0.01))
#    Conv     -> (?, 28, 28, 32)
#    Pool     -> (?, 14, 14, 32)
L1 = tf.nn.conv2d(X_img, W1, strides = [1, 1, 1, 1], padding = 'SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize = [1, 2, 2, 1],
                    strides = [1, 2, 2, 1], padding = 'SAME')

Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)  
Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)  
Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)

In [9]:
# L2 ImgIn shape=(?, 14, 14, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev = 0.01))
#    Conv      ->(?, 14, 14, 64)
#    Pool      ->(?, 7, 7, 64)
L2 = tf.nn.conv2d(L1, W2, strides = [1, 1, 1, 1], padding = 'SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize = [1, 2, 2, 1],
                    strides = [1, 2, 2, 1], padding = 'SAME')
L2_flat = tf.reshape(L2, [-1, 7 * 7 * 64])

Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)  
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)  
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)  
Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)

In [10]:
# Final FC 7x7x64 inputs -> 10 outputs
W3 = tf.get_variable("W3", shape = [7 * 7 * 64, 10],
                     initializer = tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L2_flat, W3) + b

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [11]:
# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits = logits, labels = Y
))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [12]:
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [13]:
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = int(mnist.train.num_examples / batch_size)

  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    feed_dict = {X: batch_xs, Y: batch_ys}
    c, _ = sess.run([cost, optimizer], feed_dict = feed_dict)
    avg_cost += c / total_batch

  print('Epoch:', '%04d' % (epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
Epoch: 0001 cost =  0.345545258
Epoch: 0002 cost =  0.091635759
Epoch: 0003 cost =  0.068208907
Epoch: 0004 cost =  0.056379390
Epoch: 0005 cost =  0.046861010
Epoch: 0006 cost =  0.041113833
Epoch: 0007 cost =  0.036598867
Epoch: 0008 cost =  0.032846467
Epoch: 0009 cost =  0.027880376
Epoch: 0010 cost =  0.024886485
Epoch: 0011 cost =  0.022324961
Epoch: 0012 cost =  0.020290734
Epoch: 0013 cost =  0.017093221
Epoch: 0014 cost =  0.015363020
Epoch: 0015 cost =  0.012938076
Learning Finished!


In [14]:
# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy,
                            feed_dict = {X: mnist.test.images, Y: mnist.test.labels}))

Accuracy: 0.9887


In [15]:
# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(tf.argmax(logits, 1),
                               feed_dict = {X: mnist.test.images[r:r + 1]}))

Label:  [5]
Prediction:  [5]
